<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_FastTextTests_PolEmo2_0_raw_with_pretrained_word_embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FastText benchmark for PolEmo 2.0 dataset https://clarin-pl.eu/dspace/handle/11321/710

how to use pretrianed vectors https://stackoverflow.com/questions/64706753/how-to-use-pre-trained-word-vectors-in-fasttext/64711974#64711974

In [ ]:
# install fastText
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!sudo pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 31.08 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
/content/fastText
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3082260 sha256=254e672cf5dafe78acc77a10d4f90327998692d1992b2e17c391067c0c8728f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gibzt2ce/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [ ]:
import fasttext as ft
import fasttext.util
import pandas as pd
from sklearn import model_selection
import csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# fasttext.util.download_model('pl', if_exists='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.vec.gz

In [ ]:
# !pwd

In [ ]:
# !gzip -l /content/drive/MyDrive/models/cc.pl.300.vec.gz

In [ ]:
# %ls

In [ ]:
# ft = fasttext.load_model('/content/drive/MyDrive/models/cc.pl.300.bin')


In [ ]:
# with open('/content/drive/MyDrive/models/cc.pl.300.vec','r') as file_out:
#   print(file_out.readline())
#   print(file_out.readline())
#   print(file_out.readline())
#   print(file_out.readline())
#   print(file_out.readline())
#   print(file_out.readline())


In [ ]:
# type(ft)

In [ ]:
# ft.get_dimension()

In [ ]:
# fasttext.util.reduce_model(ft, 100) failing host

In [ ]:
# ft.get_dimension()

In [ ]:
# from fastText import load_model

# # original BIN model loading
# f = load_model(YOUR-BIN-MODEL-PATH)
#     lines=[]

# get all words from model
# words = ft.get_words()

# VEC_FILE_PATH = 'cc.pl.300.created.vec'

# with open(VEC_FILE_PATH,'w') as file_out:
    
#     # the first line must contain number of total words and vector dimension
#     file_out.write(str(len(words)) + " " + str(ft.get_dimension()) + "\n")

#     # line by line, you append vectors to VEC file
#     for w in words:
#         v = ft.get_word_vector(w)
#         vstr = ""
#         for vi in v:
#             vstr += " " + str(vi)
#         try:
#             file_out.write(w + vstr+'\n')
#         except:
#             pass

In [ ]:
# TRAIN_FASTTEXT_FILE_PATH = '/content/drive/MyDrive/master_datasets/dataset_conll/hotels.text.train.txt'
# model = fasttext.train_supervised(input=TRAIN_FASTTEXT_FILE_PATH, dim=300, lr=1.0, epoch=25, pretrainedVectors='/content/drive/MyDrive/models/cc.pl.300.vec')

In [ ]:
# TEST_FASTTEXT_FILE_PATH = '/content/drive/MyDrive/master_datasets/dataset_conll/hotels.text.test.txt'
# result = model.test(TEST_FASTTEXT_FILE_PATH)

In [ ]:
# result

In [ ]:
#create work dir
WORK_PATH = '/content/data/'
%mkdir -p '/content/data/'

In [ ]:
# PATH DEFINITION FOR CORPORA
TEST_FASTTEXT_FILE_PATH = WORK_PATH + 'TEST_FASTEXT_WORK.csv'
TRAIN_FASTTEXT_FILE_PATH = WORK_PATH + 'TRAIN_FASTEXT_WORK.csv'

In [ ]:
def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      if len(content) > 0: 
        labels.append(content[-1])
        texts.append(" ".join(content[:-1]))

  # create a dataframe using texts and lables
  trainDF = pd.DataFrame()
  trainDF['text'] = texts
  trainDF['label'] = labels
  return trainDF

In [ ]:
def process_benchmark(domain, train, test, wordNgrams):
  # save text file which will be used for all tests
  test.to_csv(TEST_FASTTEXT_FILE_PATH, header=False, index=False, encoding='utf-8', sep='\t', quoting=csv.QUOTE_NONE)

  full = []
  for repeat in range(repetitions):
      iteration = []
      for n_samples in no_samples_per_class:
          train_work = train.groupby(['label']).sample(n=n_samples, replace=True)
          train_work = train_work.sample(frac=1)
          train_work.to_csv(TRAIN_FASTTEXT_FILE_PATH, header=False, index=False, encoding='utf-8', sep='\t', quoting=csv.QUOTE_NONE)
          # model = ft.train_supervised(input=TRAIN_FASTTEXT_FILE_PATH, lr=1.0, epoch=25, wordNgrams=wordNgrams)
          model = ft.train_supervised(input=TRAIN_FASTTEXT_FILE_PATH, dim=300, lr=1.0, epoch=25, pretrainedVectors='/content/drive/MyDrive/models/cc.pl.300.vec', wordNgrams=wordNgrams)
          result = model.test(TEST_FASTTEXT_FILE_PATH)
          print('domain: ', domain, " repeat: ", repeat," sample: ", n_samples,' wordNgrams: ', wordNgrams, ' results: ', result)
          iteration.append(result[1])
      print(iteration)
      full.append(iteration)
  
  return full


In [ ]:
no_samples_per_class = [1, 3, 5, 8, 10, 20, 30, 60, 100, 200] ###### FINALL SAMPLES LIST  
repetitions = 2
# grams = [1, 2]
grams = [1]


# metrice_path = '/content/drive/MyDrive/metrics/tf-idf_PolEmo2.0_logisticregression_raw' + timestamp + '.txt'
fig_path = '/content/drive/MyDrive/figures/'
dataset_path = '/content/drive/MyDrive/master_datasets/dataset_conll/'

domains = [
          #  ('all', 'MDT-A'),
          #  ('hotels', 'SDT-H'),
           ('medicine', 'SDT-M'),
           ('products', 'SDT-P'),
           ('reviews', 'SDT-R')
          ]


In [ ]:
df = pd.DataFrame()
   
for domian, ix_name in domains:
  for ngram in grams :
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%%%%%  ', domian,  ' ', ngram)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    
    CORPORA_TRAIN = dataset_path + domian + '.text.train.txt'
    CORPORA_TEST = dataset_path + domian + '.text.test.txt'
    train = load_corpora_to_dataframe(CORPORA_TRAIN)
    test = load_corpora_to_dataframe(CORPORA_TEST)

    results = process_benchmark(ix_name, train, test, ngram)

    df = df.append(pd.DataFrame(pd.DataFrame(results, columns=no_samples_per_class).mean(), columns=[ix_name + '_R_' + str(ngram)]).T)

df
### RAW

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%   all   1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
domain:  MDT-A  repeat:  0  sample:  1  wordNgrams:  1  results:  (820, 0.42317073170731706, 0.42317073170731706)
domain:  MDT-A  repeat:  0  sample:  3  wordNgrams:  1  results:  (820, 0.42317073170731706, 0.42317073170731706)
domain:  MDT-A  repeat:  0  sample:  5  wordNgrams:  1  results:  (820, 0.4036585365853659, 0.4036585365853659)
domain:  MDT-A  repeat:  0  sample:  8  wordNgrams:  1  results:  (820, 0.5414634146341464, 0.5414634146341464)
domain:  MDT-A  repeat:  0  sample:  10  wordNgrams:  1  results:  (820, 0.5121951219512195, 0.5121951219512195)
domain:  MDT-A  repeat:  0  sample:  20  wordNgrams:  1  results:  (820, 0.5719512195121951, 0.5719512195121951)
domain:  MDT-A  repeat:  0  sample:  30  wordNgrams:  1  results:  (820, 0.5073170731707317, 0.5073170731707317)
domain:  MDT-A  repeat:  0  sample:  60  wordNgrams:  1  results:  (820, 0.6158536585365854, 0.6158536585365854)
domai